# Main music quality metrics

In [ ]:
!sudo apt install -y fluidsynth

In [ ]:
!pip install --upgrade pyfluidsynth

In [ ]:
!pip install pretty_midi

In [ ]:
import numpy as np
import pretty_midi
import os
import matplotlib.pyplot as plt
from collections import Counter
import statistics
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy import stats

class MIDIQualityAnalyzer:
    """
    Клас для аналізу якості MIDI файлів, згенерованих LSTM моделлю
    Включає різні метрики музичного аналізу
    """

    def __init__(self, real_midi_folder=None, generated_midi_folder=None):
        """
        Ініціалізація аналізатора

        Args:
            real_midi_folder: папка з реальними MIDI файлами для порівняння
            generated_midi_folder: папка зі згенерованими MIDI файлами
        """
        self.real_midi_folder = real_midi_folder
        self.generated_midi_folder = generated_midi_folder
        self.real_midi_files = []
        self.generated_midi_files = []

        if real_midi_folder:
            self.load_midi_files(real_midi_folder, self.real_midi_files)
        if generated_midi_folder:
            self.load_midi_files(generated_midi_folder, self.generated_midi_files)

    def load_midi_files(self, folder_path, file_list):
        """Завантаження MIDI файлів з папки"""
        for file in os.listdir(folder_path):
            if file.endswith('.mid') or file.endswith('.midi'):
                try:
                    midi_data = pretty_midi.PrettyMIDI(os.path.join(folder_path, file))
                    file_list.append(midi_data)
                except Exception as e:
                    print(f"Помилка при завантаженні {file}: {e}")

    def add_midi_file(self, file_path, is_generated=True):
        """Додавання одного MIDI файлу для аналізу"""
        try:
            midi_data = pretty_midi.PrettyMIDI(file_path)
            if is_generated:
                self.generated_midi_files.append(midi_data)
            else:
                self.real_midi_files.append(midi_data)
            return True
        except Exception as e:
            print(f"Помилка при завантаженні {file_path}: {e}")
            return False

    def get_note_distribution(self, midi_data):
        """Отримання розподілу нот у MIDI файлі"""
        notes = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:  # Виключаємо ударні
                for note in instrument.notes:
                    notes.append(note.pitch % 12)  # Модуль 12 для отримання класу висоти
        return Counter(notes)

    def get_velocity_statistics(self, midi_data):
        """Отримання статистики по гучності (velocity) нот"""
        velocities = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    velocities.append(note.velocity)

        if not velocities:
            return {
                'mean': 0,
                'std': 0,
                'min': 0,
                'max': 0,
                'range': 0
            }

        return {
            'mean': statistics.mean(velocities),
            'std': statistics.stdev(velocities) if len(velocities) > 1 else 0,
            'min': min(velocities),
            'max': max(velocities),
            'range': max(velocities) - min(velocities)
        }

    def get_note_duration_statistics(self, midi_data):
        """Отримання статистики по тривалості нот"""
        durations = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    durations.append(note.end - note.start)

        if not durations:
            return {
                'mean': 0,
                'std': 0,
                'min': 0,
                'max': 0,
                'range': 0
            }

        return {
            'mean': statistics.mean(durations),
            'std': statistics.stdev(durations) if len(durations) > 1 else 0,
            'min': min(durations),
            'max': max(durations),
            'range': max(durations) - min(durations)
        }

    def get_polyphony_score(self, midi_data):
        """Розрахунок поліфонічного бала (скільки нот грається одночасно)"""
        piano_roll = midi_data.get_piano_roll(fs=100)
        polyphony = np.count_nonzero(piano_roll, axis=0)
        if len(polyphony) == 0:
            return {
                'mean': 0,
                'max': 0
            }
        return {
            'mean': float(np.mean(polyphony[polyphony > 0]) if np.any(polyphony > 0) else 0),
            'max': int(np.max(polyphony))
        }

    def get_harmony_consistency(self, midi_data):
        """Оцінка консистентності гармонії через аналіз акордів"""
        chords = []
        resolution = 0.25  # Чвертьнота
        end_time = midi_data.get_end_time()

        for t in np.arange(0, end_time, resolution):
            notes_at_time = []
            for instrument in midi_data.instruments:
                if not instrument.is_drum:
                    for note in instrument.notes:
                        if note.start <= t < note.end:
                            notes_at_time.append(note.pitch % 12)

            if notes_at_time:
                chords.append(frozenset(notes_at_time))

        # Підрахунок частоти акордів
        chord_counts = Counter(chords)

        # Розрахунок ентропії як міри гармонічної стабільності
        total_chords = sum(chord_counts.values())
        if total_chords == 0:
            return 0

        chord_probabilities = [count / total_chords for count in chord_counts.values()]
        entropy = -sum(p * np.log2(p) for p in chord_probabilities)

        # Визначення показника консистентності (обернено пропорційний ентропії)
        consistency = 1.0 / (1.0 + entropy) if entropy > 0 else 1.0

        return consistency

    def get_rhythmic_consistency(self, midi_data):
        """Оцінка ритмічної стабільності"""
        note_onsets = []
        for instrument in midi_data.instruments:
            if not instrument.is_drum:
                for note in instrument.notes:
                    note_onsets.append(note.start)

        if len(note_onsets) < 2:
            return 0

        # Сортування часових міток
        note_onsets.sort()

        # Розрахунок інтервалів між нотами
        intervals = [note_onsets[i+1] - note_onsets[i] for i in range(len(note_onsets)-1)]

        # Коефіцієнт варіації як міра стабільності ритму (менше - стабільніше)
        if not intervals or statistics.mean(intervals) == 0:
            return 0

        cv = statistics.stdev(intervals) / statistics.mean(intervals) if len(intervals) > 1 else 0

        # Перетворення в показник стабільності (1 - стабільний, 0 - нестабільний)
        consistency = 1.0 / (1.0 + cv)

        return consistency

    def calculate_metrics_for_file(self, midi_data):
        """Розрахунок усіх метрик для одного MIDI файлу"""
        try:
            metrics = {
                'note_duration': self.get_note_duration_statistics(midi_data),
                'velocity': self.get_velocity_statistics(midi_data),
                'polyphony': self.get_polyphony_score(midi_data),
                'harmony_consistency': self.get_harmony_consistency(midi_data),
                'rhythmic_consistency': self.get_rhythmic_consistency(midi_data),
                'duration_seconds': midi_data.get_end_time()
            }
            return metrics
        except Exception as e:
            print(f"Помилка при розрахунку метрик: {e}")
            return None

    def compare_note_distributions(self, real_dist, gen_dist):
        """Порівняння розподілу нот між реальними та згенерованими файлами"""
        all_notes = set(real_dist.keys()) | set(gen_dist.keys())
        real_probs = [real_dist.get(note, 0) for note in range(12)]
        gen_probs = [gen_dist.get(note, 0) for note in range(12)]

        # Нормалізація
        real_sum = sum(real_probs)
        gen_sum = sum(gen_probs)

        if real_sum > 0:
            real_probs = [x/real_sum for x in real_probs]
        if gen_sum > 0:
            gen_probs = [x/gen_sum for x in gen_probs]

        # KL дивергенція (з малим епсилон для уникнення ділення на 0)
        epsilon = 1e-10
        real_probs = [x + epsilon for x in real_probs]
        gen_probs = [x + epsilon for x in gen_probs]

        kl_div = sum(real_probs[i] * np.log(real_probs[i] / gen_probs[i]) for i in range(12))

        # Розрахунок косинусної подібності
        dot_product = sum(real_probs[i] * gen_probs[i] for i in range(12))
        real_norm = np.sqrt(sum(x**2 for x in real_probs))
        gen_norm = np.sqrt(sum(x**2 for x in gen_probs))

        if real_norm == 0 or gen_norm == 0:
            cosine_sim = 0
        else:
            cosine_sim = dot_product / (real_norm * gen_norm)

        return {
            'kl_divergence': kl_div,
            'cosine_similarity': cosine_sim
        }

    def analyze_generated_vs_real(self):
        """Аналіз згенерованих файлів у порівнянні з реальними"""
        if not self.real_midi_files or not self.generated_midi_files:
            print("Необхідно завантажити як реальні, так і згенеровані MIDI файли")
            return None

        # Отримання розподілу нот для всіх файлів
        real_distributions = [self.get_note_distribution(midi) for midi in self.real_midi_files]
        gen_distributions = [self.get_note_distribution(midi) for midi in self.generated_midi_files]

        # Об'єднання всіх розподілів
        combined_real_dist = Counter()
        for dist in real_distributions:
            combined_real_dist.update(dist)

        combined_gen_dist = Counter()
        for dist in gen_distributions:
            combined_gen_dist.update(dist)

        # Порівняння розподілів
        dist_comparison = self.compare_note_distributions(combined_real_dist, combined_gen_dist)

        # Розрахунок метрик для всіх файлів
        real_metrics = [self.calculate_metrics_for_file(midi) for midi in self.real_midi_files]
        real_metrics = [m for m in real_metrics if m is not None]

        gen_metrics = [self.calculate_metrics_for_file(midi) for midi in self.generated_midi_files]
        gen_metrics = [m for m in gen_metrics if m is not None]

        # Агрегація метрик
        result = {
            'note_distribution_comparison': dist_comparison,
            'metrics_comparison': {}
        }

        # Порівняння середніх значень метрик
        if real_metrics and gen_metrics:
            # Список метрик для порівняння
            metrics_to_compare = [
                ('note_duration.mean', lambda x: x['note_duration']['mean']),
                ('note_duration.std', lambda x: x['note_duration']['std']),
                ('velocity.mean', lambda x: x['velocity']['mean']),
                ('velocity.std', lambda x: x['velocity']['std']),
                ('polyphony.mean', lambda x: x['polyphony']['mean']),
                ('polyphony.max', lambda x: x['polyphony']['max']),
                ('harmony_consistency', lambda x: x['harmony_consistency']),
                ('rhythmic_consistency', lambda x: x['rhythmic_consistency']),
                ('duration_seconds', lambda x: x['duration_seconds'])
            ]

            for metric_name, extractor in metrics_to_compare:
                real_values = [extractor(m) for m in real_metrics]
                gen_values = [extractor(m) for m in gen_metrics]

                result['metrics_comparison'][metric_name] = {
                    'real_mean': statistics.mean(real_values),
                    'gen_mean': statistics.mean(gen_values),
                    'difference': statistics.mean(gen_values) - statistics.mean(real_values),
                    'difference_percent': ((statistics.mean(gen_values) / statistics.mean(real_values)) - 1) * 100 if statistics.mean(real_values) != 0 else float('inf')
                }

                # Додавання t-тесту, якщо є достатньо даних
                if len(real_values) > 1 and len(gen_values) > 1:
                    t_stat, p_value = stats.ttest_ind(real_values, gen_values, equal_var=False)
                    result['metrics_comparison'][metric_name]['t_test'] = {
                        't_statistic': t_stat,
                        'p_value': p_value,
                        'significant': p_value < 0.05
                    }

        return result

    def visualize_note_distribution(self, output_path='note_distribution.png'):
        """Візуалізація розподілу нот у реальних і згенерованих файлах"""
        if not self.real_midi_files or not self.generated_midi_files:
            print("Необхідно завантажити як реальні, так і згенеровані MIDI файли")
            return

        real_dist = Counter()
        for midi in self.real_midi_files:
            real_dist.update(self.get_note_distribution(midi))

        gen_dist = Counter()
        for midi in self.generated_midi_files:
            gen_dist.update(self.get_note_distribution(midi))

        # Нормалізація
        real_sum = sum(real_dist.values())
        gen_sum = sum(gen_dist.values())

        real_probs = [real_dist.get(i, 0) / real_sum * 100 if real_sum > 0 else 0 for i in range(12)]
        gen_probs = [gen_dist.get(i, 0) / gen_sum * 100 if gen_sum > 0 else 0 for i in range(12)]

        # Назви нот
        note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

        # Побудова графіка
        plt.figure(figsize=(12, 6))
        x = np.arange(len(note_names))
        width = 0.35

        plt.bar(x - width/2, real_probs, width, label='Реальні')
        plt.bar(x + width/2, gen_probs, width, label='Згенеровані')

        plt.xlabel('Ноти')
        plt.ylabel('Частота (%)')
        plt.title('Розподіл нот у реальних і згенерованих MIDI файлах')
        plt.xticks(x, note_names)
        plt.legend()

        plt.tight_layout()
        plt.savefig(output_path)
        plt.close()

    def visualize_metrics_comparison(self, output_path='metrics_comparison.png'):
        """Візуалізація порівняння метрик між реальними та згенерованими файлами"""
        results = self.analyze_generated_vs_real()
        if not results or 'metrics_comparison' not in results:
            print("Не вдалося отримати метрики для порівняння")
            return

        # Метрики для візуалізації
        metrics = [
            'note_duration.mean',
            'velocity.mean',
            'polyphony.mean',
            'harmony_consistency',
            'rhythmic_consistency'
        ]

        labels = [
            'Тривалість нот',
            'Гучність',
            'Поліфонія',
            'Гармонія',
            'Ритм'
        ]

        real_values = [results['metrics_comparison'][m]['real_mean'] for m in metrics]
        gen_values = [results['metrics_comparison'][m]['gen_mean'] for m in metrics]

        # Нормалізація для візуалізації
        max_values = [max(r, g) for r, g in zip(real_values, gen_values)]
        real_normalized = [r/m if m > 0 else 0 for r, m in zip(real_values, max_values)]
        gen_normalized = [g/m if m > 0 else 0 for g, m in zip(gen_values, max_values)]

        # Побудова радара
        angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
        angles += angles[:1]  # Замикання

        real_normalized += real_normalized[:1]
        gen_normalized += gen_normalized[:1]

        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
        ax.plot(angles, real_normalized, 'b-', linewidth=2, label='Реальні')
        ax.plot(angles, gen_normalized, 'r-', linewidth=2, label='Згенеровані')
        ax.fill(angles, real_normalized, 'b', alpha=0.1)
        ax.fill(angles, gen_normalized, 'r', alpha=0.1)

        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_thetagrids(np.degrees(angles[:-1]), labels)

        plt.legend(loc='upper right')
        plt.title('Порівняння музичних характеристик')
        plt.tight_layout()
        plt.savefig(output_path)
        plt.close()

    def generate_report(self, output_file='midi_analysis_report.md'):
        """Генерація звіту у форматі Markdown"""
        results = self.analyze_generated_vs_real()
        if not results:
            print("Не вдалося провести аналіз")
            return

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("# Звіт з аналізу MIDI файлів, згенерованих LSTM моделлю\n\n")

            f.write("## Основна інформація\n\n")
            f.write(f"- Кількість реальних файлів: {len(self.real_midi_files)}\n")
            f.write(f"- Кількість згенерованих файлів: {len(self.generated_midi_files)}\n\n")

            f.write("## Порівняння розподілу нот\n\n")
            f.write(f"- KL дивергенція: {results['note_distribution_comparison']['kl_divergence']:.4f} (чим ближче до 0, тим більш схожі розподіли)\n")
            f.write(f"- Косинусна подібність: {results['note_distribution_comparison']['cosine_similarity']:.4f} (від 0 до 1, де 1 - повністю збігаються)\n\n")

            f.write("## Порівняння музичних характеристик\n\n")

            f.write("| Метрика | Реальні | Згенеровані | Різниця (%) | Значущість |\n")
            f.write("|---------|----------|-----------------|-------------|------------|\n")

            for metric, data in results['metrics_comparison'].items():
                significant = data.get('t_test', {}).get('significant', '-')
                significant_str = "Так" if significant is True else "Ні" if significant is False else "-"

                f.write(f"| {metric} | {data['real_mean']:.4f} | {data['gen_mean']:.4f} | {data['difference_percent']:.2f}% | {significant_str} |\n")

            f.write("\n## Інтерпретація результатів\n\n")

            # Загальна оцінка музичної якості
            cosine_sim = results['note_distribution_comparison']['cosine_similarity']
            rhythm_match = results['metrics_comparison']['rhythmic_consistency']['difference_percent']
            harmony_match = results['metrics_comparison']['harmony_consistency']['difference_percent']

            f.write("### Загальна оцінка\n\n")

            # Тональна близькість
            if cosine_sim > 0.9:
                f.write("- **Тональна структура**: Відмінно. Розподіл нот дуже близький до оригіналу.\n")
            elif cosine_sim > 0.7:
                f.write("- **Тональна структура**: Добре. Розподіл нот достатньо близький до оригіналу.\n")
            else:
                f.write("- **Тональна структура**: Потребує покращення. Розподіл нот значно відрізняється від оригіналу.\n")

            # Ритмічна структура
            if abs(rhythm_match) < 10:
                f.write("- **Ритмічна структура**: Відмінно. Ритмічні патерни дуже близькі до оригіналу.\n")
            elif abs(rhythm_match) < 30:
                f.write("- **Ритмічна структура**: Добре. Ритмічні патерни достатньо близькі до оригіналу.\n")
            else:
                f.write("- **Ритмічна структура**: Потребує покращення. Ритмічні патерни значно відрізняються від оригіналу.\n")

            # Гармонічна структура
            if abs(harmony_match) < 10:
                f.write("- **Гармонічна структура**: Відмінно. Гармонічні послідовності дуже близькі до оригіналу.\n")
            elif abs(harmony_match) < 30:
                f.write("- **Гармонічна структура**: Добре. Гармонічні послідовності достатньо близькі до оригіналу.\n")
            else:
                f.write("- **Гармонічна структура**: Потребує покращення. Гармонічні послідовності значно відрізняються від оригіналу.\n")

            f.write("\n### Рекомендації щодо покращення моделі\n\n")

            # Формування рекомендацій на основі аналізу
            recommendations = []

            if cosine_sim < 0.7:
                recommendations.append("- Покращення розподілу нот: модель може недостатньо добре вловити тональні особливості навчальних даних.")

            if abs(results['metrics_comparison']['polyphony.mean']['difference_percent']) > 30:
                if results['metrics_comparison']['polyphony.mean']['difference_percent'] > 0:
                    recommendations.append("- Зменшення поліфонії: згенеровані файли занадто насичені нотами, що звучать одночасно.")
                else:
                    recommendations.append("- Збільшення поліфонії: згенеровані файли занадто прості з точки зору нот, що звучать одночасно.")

            if abs(results['metrics_comparison']['note_duration.mean']['difference_percent']) > 30:
                if results['metrics_comparison']['note_duration.mean']['difference_percent'] > 0:
                    recommendations.append("- Коригування тривалості нот: згенеровані ноти занадто довгі порівняно з навчальними даними.")
                else:
                    recommendations.append("- Коригування тривалості нот: згенеровані ноти занадто короткі порівняно з навчальними даними.")

            if abs(rhythm_match) > 30:
                recommendations.append("- Покращення ритмічних патернів: рекомендується використовувати додаткові функції втрат, що враховують ритмічну структуру.")

            if abs(harmony_match) > 30:
                recommendations.append("- Покращення гармонічної структури: рекомендується додати аналіз акордових послідовностей у процес навчання.")

            if recommendations:
                f.write("\n".join(recommendations))
            else:
                f.write("- Модель демонструє хорошу якість. Можливо, збільшення різноманітності навчальних даних допоможе далі покращити результати.")

        print(f"Звіт збережено у файлі {output_file}")


# Приклад використання
def main():
    # Приклад використання аналізатора
    analyzer = MIDIQualityAnalyzer(
        real_midi_folder=classical_music_midi_path + "/mozart",
        generated_midi_folder="/content"
    )

    # Аналіз і генерація звіту
    analyzer.visualize_note_distribution()
    analyzer.visualize_metrics_comparison()
    analyzer.generate_report()

    print("Аналіз завершено")

if __name__ == "__main__":
    main()

In [ ]:
!pip install seaborn music21

In [ ]:
import numpy as np
import pandas as pd
import music21 as m21
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Union
import glob


class MusicMetrics:
    """
    Клас для обчислення об'єктивних метрик якості згенерованої музики.
    """

    def __init__(self, reference_pieces: List[str] = None):
        """
        Ініціалізація класу MusicMetrics.

        Args:
            reference_pieces: список шляхів до файлів з референтними (еталонними) музичними творами
        """
        self.reference_pieces = []
        if reference_pieces:
            self.load_reference_pieces(reference_pieces)

    def load_reference_pieces(self, file_paths: List[str]):
        """
        Завантаження референтних музичних творів з файлів.

        Args:
            file_paths: список шляхів до файлів з музичними творами
        """
        for path in file_paths:
            try:
                piece = m21.converter.parse(path)
                self.reference_pieces.append(piece)
            except Exception as e:
                print(f"Помилка при завантаженні файлу {path}: {e}")

    def calculate_pitch_coherence(self, music_stream: m21.stream.Stream) -> float:
        """
        Обчислення метрики тональної послідовності (Pitch Coherence).

        Args:
            music_stream: музичний потік music21

        Returns:
            float: значення метрики від 0 до 1
        """
        # Отримуємо всі ноти в потоці
        notes = music_stream.flat.getElementsByClass(m21.note.Note)
        if len(notes) < 2:
            return 0.0

        # Обчислюємо кількість "природних" інтервалів (секунди, терції, кварти, квінти)
        natural_intervals = 0
        consonant_intervals = 0

        for i in range(len(notes) - 1):
            interval = m21.interval.Interval(notes[i], notes[i+1])
            # Інтервали до октави
            semitones = abs(interval.semitones) % 12

            # Перевіряємо, чи є інтервал "природним"
            if semitones in [1, 2, 3, 4, 5, 7]:  # секунди, терції, кварти, квінти
                natural_intervals += 1

            # Перевіряємо, чи є інтервал консонансом
            if semitones in [0, 3, 4, 5, 7, 8, 9]:  # уніссон, терції, кварти, квінти, сексти
                consonant_intervals += 1

        # Обчислюємо фінальну оцінку як середнє значення між кількістю "природних" інтервалів
        # та кількістю консонансів, нормалізоване до [0, 1]
        natural_ratio = natural_intervals / (len(notes) - 1)
        consonant_ratio = consonant_intervals / (len(notes) - 1)

        return (natural_ratio * 0.6 + consonant_ratio * 0.4)

    def calculate_rhythmic_stability(self, music_stream: m21.stream.Stream) -> float:
        """
        Обчислення метрики ритмічної стабільності (Rhythmic Stability).

        Args:
            music_stream: музичний потік music21

        Returns:
            float: значення метрики від 0 до 1
        """
        # Отримуємо всі ноти в потоці
        notes = music_stream.flat.getElementsByClass([m21.note.Note, m21.note.Rest])
        if len(notes) < 10:
            return 0.0

        # Створюємо список тривалостей нот
        durations = [note.quarterLength for note in notes]

        # Підраховуємо частоту появи кожної тривалості
        duration_counts = Counter(durations)

        # Обчислюємо ентропію розподілу тривалостей
        total_notes = len(durations)
        entropy = 0
        for count in duration_counts.values():
            probability = count / total_notes
            entropy -= probability * np.log2(probability)

        # Нормалізуємо ентропію до [0, 1] і інвертуємо (нижча ентропія = вища стабільність)
        max_entropy = np.log2(len(duration_counts))
        if max_entropy == 0:
            normalized_entropy = 0
        else:
            normalized_entropy = entropy / max_entropy

        # Обчислюємо "ритмічний патерн"
        rhythmic_patterns = []
        for i in range(len(durations) - 3):
            pattern = tuple(durations[i:i+4])
            rhythmic_patterns.append(pattern)

        # Підраховуємо повторення ритмічних патернів
        pattern_counts = Counter(rhythmic_patterns)
        total_patterns = len(rhythmic_patterns)

        if total_patterns == 0:
            pattern_repetition = 0
        else:
            # Обчислюємо відсоток повторюваних патернів
            repeated_patterns = sum(count - 1 for count in pattern_counts.values() if count > 1)
            pattern_repetition = repeated_patterns / total_patterns

        # Фінальна оцінка ритмічної стабільності
        stability = (1 - normalized_entropy) * 0.5 + pattern_repetition * 0.5

        return min(1.0, max(0.0, stability))

    def calculate_structural_integrity(self, music_stream: m21.stream.Stream) -> float:
        """
        Обчислення метрики структурної цілісності (Structural Integrity).

        Args:
            music_stream: музичний потік music21

        Returns:
            float: значення метрики від 0 до 1
        """
        # Отримуємо всі ноти в потоці
        notes = music_stream.flat.getElementsByClass(m21.note.Note)
        if len(notes) < 16:
            return 0.0

        # Розбиваємо ноти на фрази (приблизно по 8 нот у фразі)
        phrases = []
        phrase_length = 8
        for i in range(0, len(notes), phrase_length):
            if i + phrase_length <= len(notes):
                phrases.append(notes[i:i+phrase_length])

        if len(phrases) < 2:
            return 0.5  # Недостатньо фраз для аналізу структури

        # Обчислюємо показники для кожної фрази
        phrase_features = []
        for phrase in phrases:
            # Обчислюємо середню висоту нот у фразі
            avg_pitch = np.mean([note.pitch.midi for note in phrase])

            # Обчислюємо діапазон висот нот у фразі
            pitch_range = max([note.pitch.midi for note in phrase]) - min([note.pitch.midi for note in phrase])

            # Обчислюємо ритмічну різноманітність у фразі
            durations = [note.quarterLength for note in phrase]
            rhythm_diversity = len(set(durations)) / len(durations)

            phrase_features.append([avg_pitch, pitch_range, rhythm_diversity])

        # Обчислюємо косинусну подібність між сусідніми фразами
        phrase_similarities = []
        for i in range(len(phrase_features) - 1):
            similarity = cosine_similarity([phrase_features[i]], [phrase_features[i+1]])[0][0]
            phrase_similarities.append(similarity)

        # Обчислюємо середню подібність між фразами
        avg_similarity = np.mean(phrase_similarities)

        # Обчислюємо фінальну оцінку структурної цілісності
        # Використовуємо функцію з максимумом при avg_similarity = 0.7
        # (занадто висока подібність знижує структурну цілісність)
        structural_integrity = 1 - abs(avg_similarity - 0.7) / 0.7

        return min(1.0, max(0.0, structural_integrity))

    def calculate_harmonic_consistency(self, music_stream: m21.stream.Stream) -> float:
        """
        Обчислення метрики гармонічної відповідності (Harmonic Consistency).

        Args:
            music_stream: музичний потік music21

        Returns:
            float: значення метрики від 0 до 1
        """
        # Пробуємо визначити тональність
        try:
            key = music_stream.analyze('key')
        except:
            # Якщо не вдається визначити тональність, припускаємо C мажор
            key = m21.key.Key('C')

        # Отримуємо всі ноти в потоці
        notes = music_stream.flat.getElementsByClass(m21.note.Note)
        if len(notes) < 8:
            return 0.0

        # Розбиваємо ноти на групи для аналізу їх як "акорди"
        chord_groups = []
        for i in range(0, len(notes), 4):
            if i + 2 <= len(notes):
                chord_groups.append(notes[i:i+4])

        if not chord_groups:
            return 0.5

        # Підраховуємо, скільки нот належать до поточної тональності
        diatonic_notes = 0
        total_notes = len(notes)

        for note in notes:
            # Знаходимо відстань між нотою та найближчою нотою в тональності
            scale_degrees = key.getScale().pitches
            scale_midis = [p.midi % 12 for p in scale_degrees]
            note_midi = note.pitch.midi % 12

            if note_midi in scale_midis:
                diatonic_notes += 1

        diatonic_ratio = diatonic_notes / total_notes

        # Аналізуємо гармонічні переходи
        harmonic_transitions = 0
        for i in range(len(chord_groups) - 1):
            current_chord_pitches = [note.pitch.midi % 12 for note in chord_groups[i]]
            next_chord_pitches = [note.pitch.midi % 12 for note in chord_groups[i+1]]

            # Перевіряємо чи є спільні ноти між акордами
            common_notes = set(current_chord_pitches) & set(next_chord_pitches)

            if common_notes:
                harmonic_transitions += 1

            # Перевіряємо чи є рух на кварту або квінту (типові гармонічні прогресії)
            root_current = current_chord_pitches[0] if current_chord_pitches else 0
            root_next = next_chord_pitches[0] if next_chord_pitches else 0

            interval = (root_next - root_current) % 12
            if interval in [5, 7]:  # Кварта (5 півтонів) або квінта (7 півтонів)
                harmonic_transitions += 0.5

        if len(chord_groups) <= 1:
            transition_ratio = 0.5
        else:
            transition_ratio = harmonic_transitions / (len(chord_groups) - 1)
            transition_ratio = min(1.0, transition_ratio)

        # Фінальна оцінка гармонічної відповідності
        harmonic_consistency = diatonic_ratio * 0.6 + transition_ratio * 0.4

        return min(1.0, max(0.0, harmonic_consistency))

    def calculate_note_diversity(self, music_stream: m21.stream.Stream) -> float:
        """
        Обчислення різноманітності нот (Note Diversity).

        Args:
            music_stream: музичний потік music21

        Returns:
            float: значення метрики від 0 до 1
        """
        # Отримуємо всі ноти в потоці
        notes = music_stream.flat.getElementsByClass(m21.note.Note)
        if len(notes) < 8:
            return 0.0

        # Підраховуємо кількість різних висот нот
        pitches = [note.pitch.midi for note in notes]
        unique_pitches = len(set(pitches))

        # Підраховуємо кількість різних тривалостей
        durations = [note.quarterLength for note in notes]
        unique_durations = len(set(durations))

        # Обчислюємо нормалізовану різноманітність висот
        # Ідеальне значення - приблизно 12-24 унікальних нот
        pitch_diversity = min(unique_pitches / 24, 1.0)

        # Обчислюємо нормалізовану різноманітність тривалостей
        # Ідеальне значення - приблизно 4-8 унікальних тривалостей
        duration_diversity = min(unique_durations / 8, 1.0)

        # Обчислюємо ентропію розподілу висот
        pitch_counts = Counter(pitches)
        pitch_entropy = 0
        for count in pitch_counts.values():
            probability = count / len(pitches)
            pitch_entropy -= probability * np.log2(probability)

        # Нормалізуємо ентропію
        max_pitch_entropy = np.log2(unique_pitches) if unique_pitches > 0 else 0
        normalized_pitch_entropy = pitch_entropy / max_pitch_entropy if max_pitch_entropy > 0 else 0

        # Фінальна оцінка різноманітності нот
        note_diversity = pitch_diversity * 0.4 + duration_diversity * 0.3 + normalized_pitch_entropy * 0.3

        return min(1.0, max(0.0, note_diversity))

    def evaluate_piece(self, music_stream: m21.stream.Stream) -> Dict[str, float]:
        """
        Обчислення всіх метрик для музичного твору.

        Args:
            music_stream: музичний потік music21

        Returns:
            Dict[str, float]: словник з метриками
        """
        metrics = {
            "pitch_coherence": self.calculate_pitch_coherence(music_stream),
            "rhythmic_stability": self.calculate_rhythmic_stability(music_stream),
            "structural_integrity": self.calculate_structural_integrity(music_stream),
            "harmonic_consistency": self.calculate_harmonic_consistency(music_stream),
            "note_diversity": self.calculate_note_diversity(music_stream)
        }

        return metrics

    def evaluate_multiple_pieces(self, music_streams: List[m21.stream.Stream]) -> Dict[str, List[float]]:
        """
        Обчислення метрик для кількох музичних творів.

        Args:
            music_streams: список музичних потоків music21

        Returns:
            Dict[str, List[float]]: словник з метриками для кожного твору
        """
        all_metrics = {
            "pitch_coherence": [],
            "rhythmic_stability": [],
            "structural_integrity": [],
            "harmonic_consistency": [],
            "note_diversity": []
        }

        for stream in music_streams:
            metrics = self.evaluate_piece(stream)
            for key, value in metrics.items():
                all_metrics[key].append(value)

        return all_metrics

    def evaluate_lstm_attention(self, attention_model_pieces: List[m21.stream.Stream],
                               model_name: str = "LSTM + Attention") -> Dict[str, float]:
        """
        Оцінка метрик для музичних творів, створених моделлю LSTM + Attention.

        Args:
            attention_model_pieces: список музичних потоків від моделі LSTM + Attention
            model_name: назва моделі

        Returns:
            Dict[str, float]: словник із середніми значеннями метрик
        """
        attention_metrics = self.evaluate_multiple_pieces(attention_model_pieces)

        # Обчислюємо середні значення для кожної метрики
        attention_avg = {key: np.mean(values) for key, values in attention_metrics.items()}

        # Виводимо результати у вигляді таблиці
        print(f"{'Метрика':<25} | {model_name:<15}")
        print("-" * 42)

        for key in attention_avg.keys():
            print(f"{key.replace('_', ' ').title():<25} | {attention_avg[key]:.2f}")

        # Створюємо візуалізацію результатів
        plt.figure(figsize=(10, 6))

        metrics = list(attention_avg.keys())
        values = list(attention_avg.values())

        plt.bar(metrics, values, color='cornflowerblue')
        plt.ylabel('Значення метрики')
        plt.title(f'Метрики якості музики для моделі {model_name}')
        plt.xticks(rotation=30, ha='right')
        plt.tight_layout()
        plt.savefig('lstm_attention_metrics.png')
        plt.show()

        # Створюємо радіальну діаграму
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

        # Кількість метрик
        N = len(metrics)

        # Кути для кожної метрики
        angles = [n / float(N) * 2.0 * np.pi for n in range(N)]
        angles += angles[:1]  # Закриваємо коло

        # Значення метрик
        values_closed = values + [values[0]]

        # Налаштовуємо радіальну діаграму
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)

        # Встановлюємо мітки для осей
        plt.xticks(angles[:-1], [m.replace('_', ' ').title() for m in metrics])

        # Малюємо діаграму
        ax.plot(angles, values_closed, linewidth=2, linestyle='solid', label=model_name)
        ax.fill(angles, values_closed, alpha=0.25)

        plt.title(f'Об\'єктивні метрики моделі {model_name}', fontsize=15)

        plt.tight_layout()
        plt.savefig('lstm_attention_radar.png')
        plt.show()

        return attention_avg


# Приклад використання
if __name__ == "__main__":
    # Ініціалізуємо оцінювач метрик
    metrics_evaluator = MusicMetrics()

    # Шляхи до музичних творів, створених моделлю LSTM + Attention
    attention_model_files = glob.glob('/content/*.mid*', recursive=True)
    print(attention_model_files)

    # Завантажуємо музичні твори
    attention_model_pieces = []
    for file_path in attention_model_files:
        try:
            piece = m21.converter.parse(file_path)
            attention_model_pieces.append(piece)
        except Exception as e:
            print(f"Помилка при завантаженні файлу {file_path}: {e}")

    # Оцінюємо модель LSTM + Attention
    attention_metrics = metrics_evaluator.evaluate_lstm_attention(
        attention_model_pieces,
        "LSTM + Attention"
    )